In [1]:
# dependencies

import pandas as pd
import numpy as np
import csv


In [2]:
csv_schools_path = "Resources/schools_complete.csv"
csv_students_path ="Resources/students_complete.csv"

schools_complete_df = pd.read_csv(csv_schools_path)
students_complete_df = pd.read_csv(csv_students_path)
schools_complete_df.shape

(15, 5)

In [3]:
#clean up school dataframe:
schools_complete_df=schools_complete_df.drop('School ID',axis=1)
schools_clean_df = schools_complete_df.sort_values(by=['school_name'])
schools_clean_df = schools_clean_df.set_index('school_name')

#Rename column header for consistency
students_clean_df = students_complete_df.rename(columns={'Student ID': 'student_id'})

In [44]:
#Merge both csv files
pycityschools_df = pd.merge(schools_clean_df,students_clean_df,on="school_name")
pycityschools_df.head()

,school_name,type,size,budget,student_id,student_name,gender,grade,reading_score,math_score
0,Bailey High School,District,4976,3124928,17871,Blake Martin,M,9th,75,59
1,Bailey High School,District,4976,3124928,17872,Kathryn Kane,F,12th,84,58
2,Bailey High School,District,4976,3124928,17873,Richard Haas,M,11th,79,86
3,Bailey High School,District,4976,3124928,17874,Frank Marsh,M,9th,71,89
4,Bailey High School,District,4976,3124928,17875,Charles Goodman Jr.,M,9th,90,61


In [5]:
#Calculate number of schools:
school_count = len(pycityschools_df.school_name.unique())

#Calculate number of students 
    #used student_id to ensure no duplication of names
student_count = len(pycityschools_df.student_id.unique())


#Calculate total budget using original school csv file:
#school_budget=schools_clean_df.budget.unique()
total_budget = schools_clean_df.budget.sum(axis=0)

#Caculate ave scores for math and reading
ave_math_score = round(pycityschools_df.math_score.mean(),2)
ave_reading_score = round(pycityschools_df.reading_score.mean(),2)


In [54]:
#Pass/Fail determination for Math:
#pycityschools_df['math_pass_fail'] = pycityschools_df['math_score'].apply(lambda x:'P' if x>=70 else 'F')

#Pass/Fail determination for Reading:
#pycityschools_df['reading_pass_fail'] = pycityschools_df['reading_score'].apply(lambda x:'P' if x>=70 else 'F')

#Overall Pass/Fail determination - must pass math & reading
overall_pass_df = pycityschools_df.set_index("math_pass_fail")
overall_pass_list = len(overall_pass_df.loc["P","reading_pass_fail"])

#Count of students passing math:
math_pass_list= pycityschools_df.loc[pycityschools_df['math_score']>=70]['math_score'].count()
percentage_pass_math = round((math_pass_list/student_count*100),2)

#Count of students passing reading:
reading_pass_list = pycityschools_df.loc[pycityschools_df['reading_score']>=70]['reading_score'].count()
percentage_pass_reading = round((reading_pass_list/student_count*100),2)

#Count of overll passing - must pass both reading & math:
overall_pass_list = pycityschools_df[(pycityschools_df['math_score']>=70) & (pycityschools_df['reading_score']>=70)]['student_id'].count()
percentage_pass_overall = round((overall_pass_list/student_count*100),2)
overall_pass_list

25528

In [55]:
#Key metrics table - district level
district_data =[{'Total Schools':school_count,'Total Students': student_count,'Total Budget': total_budget, \
                 'Ave Math Score':ave_math_score,'Ave Reading Score':ave_reading_score,'% Passing Math':percentage_pass_math,\
                 '% Passing Reading':percentage_pass_reading,'% Overall Passing': percentage_pass_overall}]
data_district_summary_df=pd.DataFrame(district_data)
data_district_summary_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,81.88,74.98,85.81,65.17


In [ ]:
#Start school level dataset:

In [34]:
school_data_df = pycityschools_df.set_index('school_name').groupby(['school_name'])
school_type=school_data_df['type'].first()
school_student=school_data_df['student_id'].count()
school_budget=school_data_df['budget'].first()
student_budget=school_budget/school_student
school_ave_math=round(school_data_df['math_score'].mean(),2)
school_ave_reading=round(school_data_df['reading_score'].mean(),2)

school_pass_math_count=len(school_data_df[school_data_df["math_score"]>=70])
school_pass_math_count



TypeError: '>=' not supported between instances of 'SeriesGroupBy' and 'int'

In [ ]:
#calculations by schools:

